### data

In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import os

In [83]:
def load_generation_data(include_files=None, date_filter=None):
    data_dir = "/Users/jangseohyun/Documents/workspace/symply/DER/data/generation"
    all_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.csv')])

    if include_files is not None:
        for file in include_files:
            if file not in all_files:
                raise ValueError(f"파일을 찾을 수 없습니다: {file}")
        files_to_load = [f for f in all_files if f in include_files]
    else:
        files_to_load = all_files

    I = len(files_to_load)  
    T = 24 
    generation_data = np.zeros((I, T)) 

    loaded_files = []
    
    for idx, file in enumerate(files_to_load):
        file_path = os.path.join(data_dir, file)
        df = pd.read_csv(file_path)
        df.columns = df.columns.str.strip()

        date_col = "Date"
        hour_col = "Hour (Eastern Time, Daylight-Adjusted)"
        gen_col = "Electricity Generated"

        if any(col not in df.columns for col in [date_col, hour_col, gen_col]):
            print(f"{file}: 필요한 컬럼 없음. 스킵됨.")
            continue

        if date_filter:
            df = df[df[date_col] == date_filter]
            if df.empty:
                print(f"{file}: {date_filter} 데이터 없음. 스킵됨.")
                continue 

        df = df[df[hour_col].astype(str).str.match(r'^\d+$')]
        df["Time"] = df[hour_col].astype(int)
        df = df[df["Time"].between(0, 23)]

        for t in range(T):
            if t in df["Time"].values:
                generation_data[idx, t] = df[df["Time"] == t][gen_col].values[0]

        loaded_files.append(file)

    print(f"✅ 총 {I}개 파일을 불러왔습니다: {', '.join(loaded_files)}")

    return generation_data, I, T

def generate_randomized_generation(I, T, S, data, randomness_level):
    np.random.seed(1)

    noise_ranges = {
        "low": (0.8, 1.2),
        "medium": (0.5, 1.5),
        "high": (0.2, 1.8),
    }

    if randomness_level not in noise_ranges:
        raise ValueError("Invalid randomness level. Please choose 'low', 'medium', or 'high'.")

    low, high = noise_ranges[randomness_level]
    noise_factors = np.random.uniform(low, high, size=(I, T, S))

    generation_r = np.expand_dims(data, axis=-1) * noise_factors
    
    print(f"📊 데이터 Shape: I={I}, T={T}, S={S}")
    return generation_r

def plot_generation_data(generation_data, I):
    hours = np.arange(24)
    plt.figure(figsize=(15, 9))

    for i in range(I):
        plt.plot(hours, generation_data[i], marker='o', linestyle='-', alpha=0.7, label=f'Generator {i}')

    plt.xlabel("Hour")
    plt.ylabel("Electricity Generated (kWh)")
    plt.title("Hourly Electricity Generation for All Generators")
    plt.xticks(hours)  # 0~23 시간 설정
    plt.legend(loc="upper left", fontsize='small')

    plt.show()

def plot_randomized_generation(R, I, T, S):
    hours = np.arange(T)
    
    plt.figure(figsize=(15, 9))

    for i in range(I):
        plt.plot(hours, R[i, :, S], marker='o', linestyle='-', alpha=0.7, label=f'Generator {i}')

    plt.xlabel("Hour")
    plt.ylabel("Electricity Generated (kWh)")
    plt.title(f"Randomized Hourly Generation for Scenario {S}")
    plt.xticks(hours) 
    plt.legend(loc="upper left") 

    plt.show()
       
def plot_scenarios_for_generator(R, i):

    T = R.shape[1]
    S = R.shape[2] 
    hours = np.arange(T) 

    plt.figure(figsize=(15, 9))

    for s in range(S):
        plt.plot(hours, R[i, :, s], linestyle='-', alpha=0.6, label=f'Scenario {s+1}')

    plt.xlabel("Hour")
    plt.ylabel("Electricity Generated (kWh)")
    plt.title(f"Hourly Electricity Generation for Generator {i} Across All Scenarios")
    plt.xticks(hours)
    plt.legend(loc="upper left", fontsize='small', ncol=2)
    plt.show()

def generate_rt_scenarios(rt_da, S, randomness_level):

    rt_da["Time Stamp"] = pd.to_datetime(rt_da["Time Stamp"])
    nyc_rt = rt_da[rt_da["Name"] == "N.Y.C."].copy() 

    # Extract the start of the day and filter only the first 24 hours
    start_of_day = nyc_rt["Time Stamp"].min().floor("D")
    end_of_day = start_of_day + pd.Timedelta(hours=23)
    nyc_rt = nyc_rt[(nyc_rt["Time Stamp"] >= start_of_day) & (nyc_rt["Time Stamp"] <= end_of_day)]

    nyc_rt["Hour"] = nyc_rt["Time Stamp"].dt.floor("H")
    hourly_avg = nyc_rt.groupby("Hour")["LBMP ($/MWHr)"].mean().reset_index()
    price_hourly = hourly_avg["LBMP ($/MWHr)"].to_numpy()
    T = len(price_hourly)

    np.random.seed(11)
    noise_ranges = {
        "low": (0.95, 1.05),
        "medium": (0.85, 1.15),
        "high": (0.7, 1.3),
    }

    if randomness_level not in noise_ranges:
        raise ValueError("Invalid randomness level. Choose from 'low', 'medium', 'high'.")

    low, high = noise_ranges[randomness_level]
    noise_factors = np.random.uniform(low, high, size=(T, S))

    P_RT = np.expand_dims(price_hourly, axis=-1) * noise_factors

    return P_RT

def plot_rt_scenarios(P_RT):
    T, S = P_RT.shape
    hours = np.arange(T)

    plt.figure(figsize=(15, 8))

    for s in range(S):
        plt.plot(hours, P_RT[:, s], linestyle='-', alpha=0.6, label=f"Scenario {s+1}")

    plt.xlabel("Hour")
    plt.ylabel("Price ($/MWHr)")
    plt.title("Real-Time Price Scenarios (Hourly Averaged)")
    plt.xticks(hours)
    plt.legend(loc="upper left", fontsize="small", ncol=2)

    plt.show()
    
only_profit = np.array(pd.read_csv("result/result_only_profit.csv"))
ny_da = pd.read_csv("/Users/jangseohyun/Documents/workspace/symply/DER/data/price/20220718da.csv")
ny_rt = pd.read_csv("/Users/jangseohyun/Documents/workspace/symply/DER/data/price/20220718rt.csv")
ny_da["Time Stamp"] = pd.to_datetime(ny_da["Time Stamp"])
ny_da["Hour"] = ny_da["Time Stamp"].dt.hour
nyc_data = ny_da[ny_da["Name"] == "N.Y.C."]
P_DA = nyc_data["LBMP ($/MWHr)"].to_numpy() * 1.3
P_PN = P_DA * 1.5

# plot_generation_data(generation_data, 10)
# plot_randomized_generation(R, I, T, 7)
# plot_scenarios_for_generator(R, 1)
# plot_rt_scenarios(P_RT)

include_files = ['1201.csv', '137.csv', '281.csv', '397.csv', '401.csv', '430.csv', '514.csv', '524.csv', '775.csv', '89.csv']
# include_files = ['1201.csv', '137.csv', '401.csv', '524.csv', '89.csv']
generation_data, I, T = load_generation_data(include_files, "2022-07-18")

S = 50
randomness_level = "high"
R = generate_randomized_generation(I, T, S, generation_data, randomness_level)
P_RT = generate_rt_scenarios(ny_rt, S, randomness_level)


✅ 총 10개 파일을 불러왔습니다: 1201.csv, 137.csv, 281.csv, 397.csv, 401.csv, 430.csv, 514.csv, 524.csv, 775.csv, 89.csv
📊 데이터 Shape: I=10, T=24, S=50


/var/folders/2_/9m9qjphn0xs45xzpvf1hl0j00000gn/T/ipykernel_29988/3252704825.py:131: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  nyc_rt["Hour"] = nyc_rt["Time Stamp"].dt.floor("H")


### settlement model

In [84]:
set = gp.Model("Settlement")
set.setParam("PoolSolutions", 15)
set.setParam("PoolSearchMode", 2)
set.setParam("PoolGap", 0.05)

x = set.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
y_plus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
y_minus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
M_y = max(R[i, t, s] for i in range(I) for t in range(T) for s in range(S))
z_y = set.addVars(I, T, S, vtype=GRB.BINARY, name="z_y")

d = set.addVars(I, I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="d")
e_plus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="e_plus")
e_minus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="e_minus")
M_e = max(sum(R[i, t, s] for i in range(I)) for t in range(T) for s in range(S))
z_e = set.addVars(T, S, vtype=GRB.BINARY, name="z_e") 
set.update()

# obj. func.
obj = gp.quicksum(
    P_DA[t] * x[i, t] for i in range(I) for t in range(T)
) + gp.quicksum(
    1/S * (P_RT[t, s] * e_plus[i, t, s] - P_PN[t] * e_minus[i, t, s])
    for i in range(I) for t in range(T) for s in range(S)
)

set.setObjective(obj, GRB.MAXIMIZE)

Set parameter PoolSolutions to value 15
Set parameter PoolSearchMode to value 2
Set parameter PoolGap to value 0.05


In [85]:
# x로 y+, y- 결정 지어주기
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(R[i, t, s] - x[i, t] == y_plus[i, t, s] - y_minus[i, t, s])

# y+, y- 둘 중 하나는 0이어야함
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(y_plus[i, t, s] <= M_y * z_y[i, t, s])
            set.addConstr(y_minus[i, t, s] <= M_y * (1 - z_y[i, t, s]))

# rt 조건
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(y_plus[i, t, s] <= R[i, t, s])

# sum e+, sum e- 결정 지어주기
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) - gp.quicksum(x[i, t] for i in range(I)) 
                          == gp.quicksum(e_plus[i, t, s] for i in range(I)) - gp.quicksum(e_minus[i, t, s] for i in range(I)))

# sum e+, sum e- 둘 중 하나는 0이어야 함
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(gp.quicksum(e_plus[i, t, s] for i in range(I)) <= M_e * z_e[t, s])
            set.addConstr(gp.quicksum(e_minus[i, t, s] for i in range(I)) <= M_e * (1 - z_e[t, s]))

# sum e+ <= sum R
for t in range(T):
    for s in range(S):
        set.addConstr(gp.quicksum(e_plus[i, t, s] for i in range(I)) <= gp.quicksum(R[i, t, s] for i in range(I)))

# e+, e- 정의
for i in range(I):
    for t in range(T):
        for s in range(S):
            # e_plus: 초과량에서 준 양을 뺀 것
            set.addConstr(
                e_plus[i,t,s] == y_plus[i,t,s] - gp.quicksum(d[i,j,t,s] for j in range(I) if j != i)
            )
            # e_minus: 부족량에서 받은 양을 뺀 것
            set.addConstr(
                e_minus[i,t,s] == y_minus[i,t,s] - gp.quicksum(d[j,i,t,s] for j in range(I) if j != i)
            )

# 전력 이동 제한
for i in range(I):
    for t in range(T):
        for s in range(S):
            # 본인이 주는 양은 자신의 y+을 넘을 수 없음
            set.addConstr(
                gp.quicksum(d[i, j, t, s] for j in range(I) if j != i)
                <= y_plus[i, t, s]
            )
            # 본인이 받는 양은 자신의 y-을 넘을 수 없음
            set.addConstr(
                gp.quicksum(d[j, i, t, s] for j in range(I) if j != i)
                <= y_minus[i, t, s]
            )

# 자기 자신과의 거래 방지
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(d[i, i, t, s] == 0)

In [86]:
set.optimize()

if set.status == GRB.OPTIMAL:
    x_vals = np.array([[x[i, t].x for t in range(T)] for i in range(I)])
    y_plus_vals = np.array(
        [[[y_plus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    y_minus_vals = np.array(
        [[[y_minus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    d_vals = np.array(
        [
            [[[d[i, j, t, s].x for s in range(S)] for t in range(T)] for j in range(I)]
            for i in range(I)
        ]
    )
    e_plus_vals = np.array(
        [[[e_plus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    e_minus_vals = np.array(
        [[[e_minus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    print("\n- - - - - - - - - - - - - - - - - - - - - - - - -")
    print("Optimal solution found!")
    print(set.objVal)
    print(f"Optimality Gap: {set.MIPGap}")
else:
    print("\nNo optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
PoolSolutions  15
PoolSearchMode  2
PoolGap  0.05

Optimize a model with 145200 rows, 181440 columns and 1248000 nonzeros
Model fingerprint: 0xab63b495
Variable types: 168240 continuous, 13200 integer (13200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [9e-01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-02, 3e+03]
Presolve removed 105500 rows and 77250 columns
Presolve time: 0.64s
Presolved: 39700 rows, 104190 columns, 269600 nonzeros
Variable types: 90990 continuous, 13200 integer (13200 binary)
Root relaxation presolved: 39700 rows, 104190 columns, 269600 nonzeros

Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxatio

### solution pool

In [87]:
print(f"Status: {set.status}")
if set.status == GRB.OPTIMAL:
    print("모델이 Optimal입니다")

    # Solution pool 확인
    print("\nSolution Pool 정보:")
    print(f"찾은 해의 개수: {set.SolCount}")

    # Best solution의 objective value
    best_obj = set.ObjVal
    print(f"\nBest Solution의 Objective Value: {best_obj}")

    # 각 솔루션의 목적함수 값 출력 및 best solution 여부 확인
    solutions = []
    for i in range(set.SolCount):
        set.params.SolutionNumber = i
        solutions.append((i, set.PoolObjVal))
        print(f"\nSolution {i}:")
        print(f"Objective value: {set.PoolObjVal}")
        print(f"Objective bound: {set.PoolObjBound}")
        print(
            f"Is Best Solution: {abs(set.PoolObjVal - best_obj) < 1e-9}"
        )  # 수치 오차 고려

        # Gap 계산 (best solution에 대해서만)
        if abs(set.PoolObjVal - best_obj) < 1e-9:
            gap = abs(set.PoolObjVal - set.ObjBound) / abs(set.PoolObjVal)
            print(f"Gap: {gap * 100:.3f}%")

    # Solution pool 파라미터 확인
    print("\nSolution Pool 파라미터:")
    print(f"PoolSolutions: {set.Params.PoolSolutions}")
    print(f"PoolSearchMode: {set.Params.PoolSearchMode}")
    print(f"PoolGap: {set.Params.PoolGap}")
elif set.status == GRB.UNBOUNDED:
    print("모델이 Unbounded입니다")
    # Unbounded ray 확인
    set.computeIIS()
    set.write("model.ilp")
elif set.status == GRB.INFEASIBLE:
    print("모델이 Infeasible입니다")
    # IIS 계산
    set.computeIIS()
    set.write("model.ilp")
elif set.status == GRB.INF_OR_UNBD:
    print("모델이 Infeasible 또는 Unbounded입니다")
    # Presolve를 끄고 다시 시도
    set.setParam('Presolve', 0)
    set.optimize()
elif set.status == GRB.TIME_LIMIT:
    print("시간 제한에 도달했습니다")
    if set.SolCount > 0:
        print(f"현재까지의 최선해: {set.objVal}")
        print(f"Optimality Gap: {set.MIPGap}")

Status: 2
모델이 Optimal입니다

Solution Pool 정보:
찾은 해의 개수: 15

Best Solution의 Objective Value: 2124734.377772276

Solution 0:
Objective value: 2124734.3777722763
Objective bound: 2124734.377772276
Is Best Solution: True
Gap: 0.000%

Solution 1:
Objective value: 2124734.377772276
Objective bound: 2124734.377772276
Is Best Solution: True
Gap: 0.000%

Solution 2:
Objective value: 2124734.377772276
Objective bound: 2124734.377772276
Is Best Solution: True
Gap: 0.000%

Solution 3:
Objective value: 2124734.377772276
Objective bound: 2124734.377772276
Is Best Solution: True
Gap: 0.000%

Solution 4:
Objective value: 2124734.377772276
Objective bound: 2124734.377772276
Is Best Solution: True
Gap: 0.000%

Solution 5:
Objective value: 2124734.377772276
Objective bound: 2124734.377772276
Is Best Solution: True
Gap: 0.000%

Solution 6:
Objective value: 2124734.377772276
Objective bound: 2124734.377772276
Is Best Solution: True
Gap: 0.000%

Solution 7:
Objective value: 2124734.377772276
Objective bound: 

### 결과 분석

#### 수익 분석

In [88]:
# Day-ahead 수익 비교
total_da_profit_obj = 0
for i in range(I):
    for t in range(T):
        total_da_profit_obj += P_DA[t] * x[i, t].x

# Real-time 수익 비교
total_rt_profit_obj = 0
for i in range(I):
    for t in range(T):
        for s in range(S):
            rt_profit_obj = P_RT[t, s] * e_plus[i, t, s].x
            total_rt_profit_obj += 1 / S * rt_profit_obj

# 패널티 비용 비교
total_penalty_cost_obj = 0
for i in range(I):
    for t in range(T):
        for s in range(S):
            penalty_cost_obj = P_PN[t] * e_minus[i, t, s].x
            total_penalty_cost_obj += 1 / S * penalty_cost_obj

# 총 시스템 이익 (목적 함수 기반)
total_system_profit_obj = (
    total_da_profit_obj + total_rt_profit_obj - total_penalty_cost_obj
)

# 결과 출력
print(f"DA: {total_da_profit_obj:.2f}")
print(f"RT: {total_rt_profit_obj:.2f}")
print(f"Penalty: {total_penalty_cost_obj:.2f}")
print(f"목적 함수 기반 총 이익 (_obj): {total_system_profit_obj:.2f}")

DA: 587828.00
RT: 1557001.62
Penalty: 20095.25
목적 함수 기반 총 이익 (_obj): 2124734.38


#### 하루 aggregated 커밋량 분석

In [89]:
for t in range(T):
    avg_e_plus = sum(e_plus_vals[i, t, s] for i in range(I) for s in range(S)) / S
    avg_e_minus = sum(e_minus_vals[i, t, s] for i in range(I) for s in range(S)) / S
    print(f"[시간 {t}] x: {sum(x_vals[i, t] for i in range(I)):.3f}, e+: {avg_e_plus:.3f}, e-: {avg_e_minus:.3f}")

total_x = sum(x_vals[i, t] for i in range(I) for t in range(T))
print(f"총 하루 commitment: {total_x:.3f}")

[시간 0] x: 0.000, e+: 0.000, e-: 0.000
[시간 1] x: 0.000, e+: 0.111, e-: 0.000
[시간 2] x: 0.069, e+: 0.179, e-: 0.001
[시간 3] x: 0.000, e+: 0.000, e-: 0.000
[시간 4] x: 0.388, e+: 0.329, e-: 0.010
[시간 5] x: 0.000, e+: 1.002, e-: 0.000
[시간 6] x: 6.359, e+: 2.196, e-: 0.256
[시간 7] x: 23.282, e+: 21.642, e-: 0.123
[시간 8] x: 76.576, e+: 26.075, e-: 1.057
[시간 9] x: 268.058, e+: 34.761, e-: 10.341
[시간 10] x: 602.568, e+: 115.804, e-: 8.922
[시간 11] x: 0.000, e+: 900.061, e-: 0.000
[시간 12] x: 0.000, e+: 1255.613, e-: 0.000
[시간 13] x: 0.000, e+: 1822.463, e-: 0.000
[시간 14] x: 1637.641, e+: 306.228, e-: 34.951
[시간 15] x: 0.000, e+: 1168.557, e-: 0.000
[시간 16] x: 785.306, e+: 171.277, e-: 15.127
[시간 17] x: 0.000, e+: 867.622, e-: 0.000
[시간 18] x: 0.000, e+: 669.892, e-: 0.000
[시간 19] x: 388.340, e+: 89.799, e-: 14.077
[시간 20] x: 119.866, e+: 17.994, e-: 4.812
[시간 21] x: 23.335, e+: 5.017, e-: 1.540
[시간 22] x: 0.000, e+: 0.000, e-: 0.000
[시간 23] x: 0.000, e+: 0.000, e-: 0.000
총 하루 commitment: 3931.789


#### exchange process

In [90]:
for t in range(13,15):
    x_sum = sum(x_vals[i, t] for i in range(I))

    for s in range(S):
        y_plus_sum = sum(y_plus_vals[i, t, s] for i in range(I))
        y_minus_sum = sum(y_minus_vals[i, t, s] for i in range(I))

        e_plus_sum = sum(e_plus_vals[i, t, s] for i in range(I))
        e_minus_sum = sum(e_minus_vals[i, t, s] for i in range(I))

        R_sum = sum(R[i, t, s] for i in range(I))

        print(f"[t={t} s={s}]")

        print(f"x 합계: {x_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {x_vals[i,t]:.2f}", end=" ")
        print(")")

        print(f"R 합계: {R_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {R[i, t, s]:.2f}", end=" ")
        print(")")

        print(f"y+ 합계: {y_plus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {y_plus_vals[i,t,s]:.2f}", end=" ")
        print(")")

        print(f"y- 합계: {y_minus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {y_minus_vals[i,t,s]:.2f}", end=" ")
        print(")")

        print(f"e+ 합계: {e_plus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {e_plus_vals[i,t,s]:.2f}", end=" ")
        print(")")

        print(f"e- 합계: {e_minus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {e_minus_vals[i,t,s]:.2f}", end=" ")
        print(")")

        print("거래 내역:")
        for i in range(I):
            for j in range(I):
                if d_vals[i, j, t, s] > 1e-6:
                    print(
                        f"발전기 {i}가 발전기 {j}에게 {d_vals[i,j,t,s]:.2f} 전력을 줌"
                    )
        print()

[t=13 s=0]
x 합계: 0.00 ([0] 0.00 [1] 0.00 [2] 0.00 [3] 0.00 [4] 0.00 [5] 0.00 [6] 0.00 [7] 0.00 [8] 0.00 [9] 0.00 )
R 합계: 1634.85 ([0] 156.57 [1] 296.19 [2] 62.67 [3] 72.85 [4] 333.40 [5] 51.40 [6] 58.74 [7] 427.94 [8] 133.47 [9] 41.62 )
y+ 합계: 1634.85 ([0] 156.57 [1] 296.19 [2] 62.67 [3] 72.85 [4] 333.40 [5] 51.40 [6] 58.74 [7] 427.94 [8] 133.47 [9] 41.62 )
y- 합계: 0.00 ([0] 0.00 [1] 0.00 [2] 0.00 [3] 0.00 [4] 0.00 [5] 0.00 [6] 0.00 [7] 0.00 [8] 0.00 [9] 0.00 )
e+ 합계: 1634.85 ([0] 156.57 [1] 296.19 [2] 62.67 [3] 72.85 [4] 333.40 [5] 51.40 [6] 58.74 [7] 427.94 [8] 133.47 [9] 41.62 )
e- 합계: 0.00 ([0] 0.00 [1] 0.00 [2] 0.00 [3] 0.00 [4] 0.00 [5] 0.00 [6] 0.00 [7] 0.00 [8] 0.00 [9] 0.00 )
거래 내역:

[t=13 s=1]
x 합계: 0.00 ([0] 0.00 [1] 0.00 [2] 0.00 [3] 0.00 [4] 0.00 [5] 0.00 [6] 0.00 [7] 0.00 [8] 0.00 [9] 0.00 )
R 합계: 1996.55 ([0] 75.70 [1] 443.30 [2] 224.89 [3] 126.36 [4] 709.06 [5] 55.48 [6] 83.79 [7] 171.77 [8] 45.45 [9] 60.76 )
y+ 합계: 1996.55 ([0] 75.70 [1] 443.30 [2] 224.89 [3] 126.36 [4]

In [91]:
d_given = np.zeros(I)
d_received = np.zeros(I)

for i in range(I):
    d_given[i] = sum(
        sum(d_vals[i, j, t, s] for j in range(I) if j != i for t in range(T)) / S for s in range(S)
    )
    d_received[i] = sum(
        sum(d_vals[j, i, t, s] for j in range(I) if j != i for t in range(T)) / S for s in range(S)
    )

set_d = pd.DataFrame({"set_d_given": d_given, "set_d_received": d_received})
set_d.to_csv("result/result_set1_d.csv", index=False)

### 정산

In [92]:
total_der_profit = 0
der_profit = {}
for i in range(I):
    der_profit[i] = sum(P_DA[t] * x[i,t].x + sum(1/S * (P_RT[t,s] * e_plus[i,t,s].x - P_PN[t] * e_minus[i,t,s].x) for s in range(S)) for t in range(T))
    total_der_profit += der_profit[i]

print(f"\n모든 der의 profit 합계: {total_der_profit:.2f}")

print("\n최종 profit:")
for i, (profit, only) in enumerate(zip(der_profit.values(), only_profit.flatten())):
    increase_percentage = ((profit - only) / only) * 100
    print(f"[{i}] {profit:.2f} ({increase_percentage:.2f}%)")


모든 der의 profit 합계: 2124734.38

최종 profit:
[0] 204158.32 (1.89%)
[1] 336579.70 (1.31%)
[2] 196210.94 (2.30%)
[3] 57161.38 (15.80%)
[4] 397554.70 (-0.32%)
[5] 97244.86 (7.63%)
[6] 99067.49 (10.06%)
[7] 436595.93 (-1.73%)
[8] 131039.97 (2.37%)
[9] 169121.10 (5.77%)


### 결과 저장

In [93]:
set_profit_value = pd.DataFrame({'set_profit_value': der_profit})
set_profit_value.to_csv('result/result_set1_profit.csv', index=False)

set_obj = pd.DataFrame({'set_obj': [set.objVal]})
set_obj.to_csv('result/result_set1_obj.csv', index=False)